# **Challenge 2**

In [1]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

***Challenge 2***

You may use the same testbed but without the constraints on external datasets or models trained on exeternal datasets. You may not however use any of the CIFAR-10 training set.

# **ResNet18**

In [4]:
import torchvision.models as models
import numpy as np
import torch
from numpy.random import RandomState
import torch.optim as optim
from torch.utils.data import Subset
import time
from torchvision import datasets, transforms

# Normalization to match ImageNet statistics
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Resize images to match the input size required by ResNet
resize = transforms.Resize((224, 224))

transform_val = transforms.Compose([resize, transforms.ToTensor(), normalize])
transform_train = transforms.Compose([resize, transforms.ToTensor(), normalize])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# CIFAR Data preparation
cifar_data = datasets.CIFAR10(root='.', train=True, transform=transform_train, download=True)
cifar_data_val = datasets.CIFAR10(root='.', train=True, transform=transform_val, download=True)

accs = []
times = []


for seed in range(1, 5):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 5000))
    classes = prng.permutation(np.arange(0,10))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d' % (len(train_data.indices), len(val_data.indices)))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    # Load ResNet18
    model = models.resnet18(pretrained=True)

    # Freeze all layers in the model
    for param in model.parameters():
        param.requires_grad = False

    # Replace the classifier
    num_features = model.fc.in_features
    model.fc = torch.nn.Linear(num_features, 10)

    # Optimizer for the classifier
    optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

    model.to(device)

    start_time = time.time()
    for epoch in range(10):
        start_time = time.time()

        train(model, device, train_loader, optimizer, epoch, display=True)

    end_time = time.time()
    times.append(end_time - start_time)
    accs.append(test(model, device, val_loader))

accs = np.array(accs)
times = np.array(times)

print('Acc over 2 instances: %.2f +- %.2f' % (accs.mean(), accs.std()))
print(f"Average Time over 5 instances: {times.mean()} +- {times.std()}")

cuda


100%|██████████| 170498071/170498071 [00:03<00:00, 43483603.99it/s]


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 141MB/s]


Train Epoch: 0 [0/50 (0%)]	Loss: 2.485786
Train Epoch: 1 [0/50 (0%)]	Loss: 1.072920
Train Epoch: 2 [0/50 (0%)]	Loss: 0.542311
Train Epoch: 3 [0/50 (0%)]	Loss: 0.388291
Train Epoch: 4 [0/50 (0%)]	Loss: 0.267271
Train Epoch: 5 [0/50 (0%)]	Loss: 0.202988
Train Epoch: 6 [0/50 (0%)]	Loss: 0.141132
Train Epoch: 7 [0/50 (0%)]	Loss: 0.102156
Train Epoch: 8 [0/50 (0%)]	Loss: 0.081247
Train Epoch: 9 [0/50 (0%)]	Loss: 0.064737


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1277, Accuracy: 380/400 (95.00%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.259145
Train Epoch: 1 [0/50 (0%)]	Loss: 1.047936
Train Epoch: 2 [0/50 (0%)]	Loss: 0.530387
Train Epoch: 3 [0/50 (0%)]	Loss: 0.330076
Train Epoch: 4 [0/50 (0%)]	Loss: 0.208290
Train Epoch: 5 [0/50 (0%)]	Loss: 0.138203
Train Epoch: 6 [0/50 (0%)]	Loss: 0.097382
Train Epoch: 7 [0/50 (0%)]	Loss: 0.073447
Train Epoch: 8 [0/50 (0%)]	Loss: 0.058715
Train Epoch: 9 [0/50 (0%)]	Loss: 0.048566

Test set: Average loss: 0.0931, Accuracy: 384/400 (96.00%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.584681
Train Epoch: 1 [0/50 (0%)]	Loss: 1.158858
Train Epoch: 2 [0/50 (0%)]	Loss: 0.570427
Train Epoch: 3 [0/50 (0%)]	Loss: 0.443393
Train Epoch: 4 [0/50 (0%)]	Loss: 0.311887
Train Epoch: 5 [0/50 (0%)]	Loss: 0.252818
Train Epoch: 6 [0/50 (0%)]	Loss: 0.188937
Train Epoch: 7 [0/50 (0%)]	Loss: 0.140801
Train Epoch: 8 [0/

# **ResNet50**

In [5]:
import torchvision.models as models
import numpy as np
import torch
from numpy.random import RandomState
import torch.optim as optim
from torch.utils.data import Subset
import time
from torchvision import datasets, transforms

# Normalization to match ImageNet statistics
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Resize images to match the input size required by ResNet
resize = transforms.Resize((224, 224))

transform_val = transforms.Compose([resize, transforms.ToTensor(), normalize])
transform_train = transforms.Compose([resize, transforms.ToTensor(), normalize])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

# CIFAR Data preparation
cifar_data = datasets.CIFAR10(root='.', train=True, transform=transform_train, download=True)
cifar_data_val = datasets.CIFAR10(root='.', train=True, transform=transform_val, download=True)

accs = []
times = []


for seed in range(1, 5):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 5000))
    classes = prng.permutation(np.arange(0,10))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d' % (len(train_data.indices), len(val_data.indices)))

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

    # Load ResNet18
    model = models.resnet50(pretrained=True)

    # Freeze all layers in the model
    for param in model.parameters():
        param.requires_grad = False

    # Replace the classifier
    num_features = model.fc.in_features
    model.fc = torch.nn.Linear(num_features, 10)

    # Optimizer for the classifier
    optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

    model.to(device)

    start_time = time.time()
    for epoch in range(10):
        start_time = time.time()

        train(model, device, train_loader, optimizer, epoch, display=True)

    end_time = time.time()
    times.append(end_time - start_time)
    accs.append(test(model, device, val_loader))

accs = np.array(accs)
times = np.array(times)

print('Acc over 2 instances: %.2f +- %.2f' % (accs.mean(), accs.std()))
print(f"Average Time over 5 instances: {times.mean()} +- {times.std()}")

cuda
Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 130MB/s]


Train Epoch: 0 [0/50 (0%)]	Loss: 2.385369
Train Epoch: 1 [0/50 (0%)]	Loss: 1.001565
Train Epoch: 2 [0/50 (0%)]	Loss: 0.547900
Train Epoch: 3 [0/50 (0%)]	Loss: 0.414374
Train Epoch: 4 [0/50 (0%)]	Loss: 0.316099
Train Epoch: 5 [0/50 (0%)]	Loss: 0.243793
Train Epoch: 6 [0/50 (0%)]	Loss: 0.185166
Train Epoch: 7 [0/50 (0%)]	Loss: 0.150144
Train Epoch: 8 [0/50 (0%)]	Loss: 0.120273
Train Epoch: 9 [0/50 (0%)]	Loss: 0.096210


/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1005, Accuracy: 393/400 (98.25%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.353316
Train Epoch: 1 [0/50 (0%)]	Loss: 0.966390
Train Epoch: 2 [0/50 (0%)]	Loss: 0.494506
Train Epoch: 3 [0/50 (0%)]	Loss: 0.349339
Train Epoch: 4 [0/50 (0%)]	Loss: 0.251916
Train Epoch: 5 [0/50 (0%)]	Loss: 0.182876
Train Epoch: 6 [0/50 (0%)]	Loss: 0.135799
Train Epoch: 7 [0/50 (0%)]	Loss: 0.103523
Train Epoch: 8 [0/50 (0%)]	Loss: 0.080947
Train Epoch: 9 [0/50 (0%)]	Loss: 0.064715

Test set: Average loss: 0.0337, Accuracy: 399/400 (99.75%)

Num Samples For Training 50 Num Samples For Val 400
Train Epoch: 0 [0/50 (0%)]	Loss: 2.305602
Train Epoch: 1 [0/50 (0%)]	Loss: 1.036864
Train Epoch: 2 [0/50 (0%)]	Loss: 0.645875
Train Epoch: 3 [0/50 (0%)]	Loss: 0.453728
Train Epoch: 4 [0/50 (0%)]	Loss: 0.387158
Train Epoch: 5 [0/50 (0%)]	Loss: 0.277740
Train Epoch: 6 [0/50 (0%)]	Loss: 0.216986
Train Epoch: 7 [0/50 (0%)]	Loss: 0.190444
Train Epoch: 8 [0/